# Assignment 2

In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install numpy
#!pip install matplotlib

In [2]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, make_scorer, cohen_kappa_score, matthews_corrcoef, log_loss
from sklearn.model_selection import StratifiedKFold
import json

In [3]:
# Read in data
df = pd.read_csv('./forestCover.csv', na_values='?')
display(df.head())
print(df.shape)

# Convert positive in Soil_Type1 to 1 and negativr to 0
df["Soil_Type1"] = (df["Soil_Type1"] == "positive").astype(int)

,Elevation,Aspect,Facet,Slope,Inclination,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Water_Level,Observation_ID,Cover_Type
0,3208920,166,415.394727,6.0,0.691628,408,62,3137,225,242,...,0,0,0,0,0,0,0,2,1,2
1,2789020,137,343.302186,9.0,0.621245,30,2,1040,235,237,...,0,0,0,0,0,0,0,3,1,3
2,3384615,357,894.231390,9.0,-0.266086,362,15,3113,207,225,...,0,0,0,0,0,0,0,4,1,1
3,3348150,148,371.346939,6.0,0.786375,247,50,1207,228,240,...,0,0,0,0,0,0,0,5,1,2
4,3061955,124,310.783430,11.0,-0.335685,170,19,1595,238,232,...,0,0,0,0,0,0,0,6,1,2


(581012, 59)


In [4]:
# Count missing values
missing_values_count = df.isnull().sum()
print(missing_values_count)

# slope
print()
print(df['Slope'].describe())

# Conclusion - seems to be basically no missing values except for small amount for Slope
# Plan to impute with median as seems to have high outliers.

Elevation                               0
Aspect                                  0
Facet                                   0
Slope                                 298
Inclination                             0
Horizontal_Distance_To_Hydrology        0
Vertical_Distance_To_Hydrology          0
Horizontal_Distance_To_Roadways         0
Hillshade_9am                           0
Hillshade_Noon                          0
Hillshade_3pm                           0
Horizontal_Distance_To_Fire_Points      0
Wilderness_Area1                        0
Wilderness_Area2                        0
Wilderness_Area3                        0
Wilderness_Area4                        0
Soil_Type1                              0
Soil_Type2                              0
Soil_Type3                              0
Soil_Type4                              0
Soil_Type5                              0
Soil_Type6                              0
Soil_Type7                              0
Soil_Type8                        

### Data cleaning:
**Classification trees data fixes:**
- Skew distribution
- Category and numerical (One hot encoding?)

Robust against
- Outliers
- Noise
- Missing values
  

**KNN data fixes:**
- Normalise
- Remove missing

Robust agains
- outliers

**Fixes foor both**
- Remove one with only 1 cardinality
- Remove correlated features
- Remove id

In [5]:
# Remove rows with any missing values
df = df.dropna()

print(df["Aspect"].corr(df["Facet"]))
print(df["Aspect"].corr(df["Cover_Type"]))
print(df["Facet"].corr(df["Cover_Type"]))

# Facet is slightly more correlated to target than Aspect

df = df.drop(columns=['Observation_ID', 'Water_Level', 'Aspect'], axis = 1)

0.9999980537370698
0.017068499402923768
0.01707189190902871


In [6]:
X = df.drop(columns=['Cover_Type'], axis = 1)
y = df['Cover_Type']

## Classification trees

In [ ]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'cohen_kappa': make_scorer(cohen_kappa_score),
    'matthews_corrcoef': make_scorer(matthews_corrcoef)
}

# Stratified 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# deviations to test
depth_values = [5, 10, 15, 20, 25, 55] 
criterion_values = ["gini", "entropy", "log_loss"] 

cv_results_dict_dt = {}

for criterion in criterion_values:
    for depth in depth_values:
        print(f"Evaluating Decision Tree with criterion = {criterion} and max_depth = {depth}")
        
        tree = DecisionTreeClassifier(criterion= criterion, max_depth = depth, random_state = 1)

        # stratified cross fold - for imbalance target values
        cv_results = cross_validate(tree, X, y, cv = skf, scoring = scoring, return_train_score = True)

        # save the cross fold results
        if criterion not in cv_results_dict_dt:
            cv_results_dict_dt[criterion] = {}
        cv_results_dict_dt[criterion][depth] = {key: cv_results[key].tolist() for key in cv_results.keys()}


        print(f"Mean Training Accuracy: {np.mean(cv_results['train_accuracy'])} and std {np.std(cv_results['train_accuracy'])}")
        print(f"Mean Test Accuracy: {np.mean(cv_results['test_accuracy'])} and std {np.std(cv_results['test_accuracy'])}")
        print(f"Mean Training F1-score: {np.mean(cv_results['train_f1_macro'])} and std {np.std(cv_results['train_f1_macro'])}")
        print(f"Mean Test F1-score: {np.mean(cv_results['test_f1_macro'])} and std {np.std(cv_results['test_f1_macro'])}")
        print(f"Mean Training Cohen's Kappa: {np.mean(cv_results['train_cohen_kappa'])} and std {np.std(cv_results['train_cohen_kappa'])}")
        print(f"Mean Test Cohen's Kappa: {np.mean(cv_results['test_cohen_kappa'])} and std {np.std(cv_results['test_cohen_kappa'])}")
        print(f"Mean Training Matthews Correlation Coefficient: {np.mean(cv_results['train_matthews_corrcoef'])} and std {np.std(cv_results['train_matthews_corrcoef'])}")
        print(f"Mean Test Matthews Correlation Coefficient: {np.mean(cv_results['test_matthews_corrcoef'])} and std {np.std(cv_results['test_matthews_corrcoef'])}")
        print("-" * 30)

# save 
with open("decision_tree_cv_results.txt", "w") as f:
    json.dump(cv_results_dict_dt, f, indent=4)


Evaluating Decision Tree with criterion = gini and max_depth = 5
Mean Training Accuracy: 0.7032596942478311 and std 0.00038575431254808077
Mean Test Accuracy: 0.7024232242864963 and std 0.0016857929783280908
Mean Training F1-score: 0.4802539876582966 and std 0.003749621729695445
Mean Test F1-score: 0.47863112025705 and std 0.004695568584433108
Mean Training Cohen's Kappa: 0.5123715527534476 and std 0.00211311421409935
Mean Test Cohen's Kappa: 0.5110489945253122 and std 0.0035203436261267477
Mean Training Matthews Correlation Coefficient: 0.5137595243893482 and std 0.0024571148939705866
Mean Test Matthews Correlation Coefficient: 0.5124414021356083 and std 0.0038193966576298174
------------------------------
Evaluating Decision Tree with criterion = gini and max_depth = 10
Mean Training Accuracy: 0.7820316886321005 and std 0.0008377330472541606
Mean Test Accuracy: 0.7779044420346859 and std 0.0016795736696696673
Mean Training F1-score: 0.6725377505287261 and std 0.007013622990980536
Mea

### Results: Classification

Best is depth = 25, homogeneity = entropy

Now want to see what those features were

In [ ]:
tree = DecisionTreeClassifier(criterion= "entropy", max_depth = 25, random_state = 1)
tree.fit(X,y)

feature_importance = tree.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print(feature_importance_df)

### KNN
Data cleaning for knn

In [ ]:
# impute slope missing values with median
X['Slope'].fillna( df['Slope'].median())

Feature selection
- The soil types merge like they did in orginal paper
- Those not correlated at all with cover_type?
- Not choosen by the decision tree

In [ ]:
# 1-6
# 7-8
# 9 
# 10 -13
# 14-15
# 16-17
# 18
# 19-21
# 22-23
# 24-31  

grouped_soil_type = X.copy()

grouped_soil_type['Soil_Type_1_to_6'] = grouped_soil_type[['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6']].max(axis=1)
grouped_soil_type['Soil_Type_7_to_8'] = grouped_soil_type[['Soil_Type7', 'Soil_Type8']].max(axis=1)
grouped_soil_type['Soil_Type_9'] = grouped_soil_type['Soil_Type9']
grouped_soil_type['Soil_Type_10_to_13'] = grouped_soil_type[['Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13']].max(axis=1)
grouped_soil_type['Soil_Type_14_to_15'] = grouped_soil_type[['Soil_Type14', 'Soil_Type15']].max(axis=1)
grouped_soil_type['Soil_Type_16_to_17'] = grouped_soil_type[['Soil_Type16', 'Soil_Type17']].max(axis=1)
grouped_soil_type['Soil_Type_18'] = grouped_soil_type['Soil_Type18']
grouped_soil_type['Soil_Type_19_to_21'] = grouped_soil_type[['Soil_Type19', 'Soil_Type20', 'Soil_Type21']].max(axis=1)
grouped_soil_type['Soil_Type_22_to_23'] = grouped_soil_type[['Soil_Type22', 'Soil_Type23']].max(axis=1)
grouped_soil_type['Soil_Type_24_to_31'] = grouped_soil_type[['Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31']].max(axis=1)


soil_type_to_drop = [col for col in X.columns if col.startswith('Soil_Type') and not 'Soil_Type_' in col]
grouped_soil_type = grouped_soil_type.drop(columns=soil_type_to_drop)

display(grouped_soil_type.head())

In [ ]:
# Normalise
X.drop(columns = ['Inclination'], axis = 1 )
scaler = StandardScaler()
X_normalised = scaler.fit_transform(grouped_soil_type)

k = 763  (square root of observations)
Distance measures - Euclidean,

In [ ]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'cohen_kappa': make_scorer(cohen_kappa_score),
    'matthews_corrcoef': make_scorer(matthews_corrcoef)
}

# range of k
k_values = [1, 10, 100, 763]

# Stratified cv
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

cv_results_dict = {}

for k in k_values:
    print(f"Evaluating KNN with k = {k}")

    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs = -1, metric = "manhattan") #bit of a patch job - change metric to either euclidean or cosine if needed

    #  5-fold cross-validation
    cv_results = cross_validate(knn, X_train, y_train, cv=skf, scoring=scoring, return_train_score=True)

    cv_results_dict[k] = cv_results

    print(f"Mean Training Accuracy: {np.mean(cv_results['train_accuracy'])} and std {np.std(cv_results['train_accuracy'])}")
    print(f"Mean Test Accuracy: {np.mean(cv_results['test_accuracy'])} and std {np.std(cv_results['test_accuracy'])}")
    print(f"Mean Training F1-score: {np.mean(cv_results['train_f1_macro'])} and std {np.std(cv_results['train_f1_macro'])}")
    print(f"Mean Test F1-score: {np.mean(cv_results['test_f1_macro'])} and std {np.std(cv_results['test_f1_macro'])}")
    print(f"Mean Training Cohen's Kappa: {np.mean(cv_results['train_cohen_kappa'])} and std {np.std(cv_results['train_cohen_kappa'])}")
    print(f"Mean Test Cohen's Kappa: {np.mean(cv_results['test_cohen_kappa'])} and std {np.std(cv_results['test_cohen_kappa'])}")
    print(f"Mean Training Matthews Correlation Coefficient: {np.mean(cv_results['train_matthews_corrcoef'])} and std {np.std(cv_results['train_matthews_corrcoef'])}")
    print(f"Mean Test Matthews Correlation Coefficient: {np.mean(cv_results['test_matthews_corrcoef'])} and std {np.std(cv_results['test_matthews_corrcoef'])}")
    print("-" * 30)

In [ ]:
print(cv_results_dict)